---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [2]:
import pandas as pd
import numpy as np
import re

def clean(item):
    if re.search("[0-9]+", item):
        spos = re.search("[0-9]+", item).start()
        return item[:spos]
    if re.search("\(.*\)", item):
        spos = re.search("\(.*\)", item).start()-1
        return item[:spos]
    return item

def renamecountry(item):
    item = clean(item)
    if(item == "Republic of Korea"):
        return "South Korea"
    if(item == "United States of America"):
        return "United States"
    if(item == "United Kingdom of Great Britain and Northern Ireland"):
        return "United Kingdom"
    if(item == "China, Hong Kong Special Administrative Region"):
        return "Hong Kong"
    return item

def grenamecountry(item):
    if(item == "Korea, Rep."):
        return "South Korea"
    if(item == "Iran, Islamic Rep."):
        return "Iran"
    if(item == "Hong Kong SAR, China"):
        return "Hong Kong"
    return item

def ans():
    energy = pd.read_excel('Energy Indicators.xls', skiprows = 17,  skipfooter = 38)
    energy = energy.drop(energy.columns[[0,1]], axis = 1)
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy.replace(to_replace = "...", value = np.NaN, inplace = True)
    energy['Energy Supply'] = energy.apply(lambda x : (x['Energy Supply'] * 1000000), axis = 1)
    energy['Country'] = energy['Country'].apply(renamecountry)
    
    GDP = pd.read_csv('world_bank.csv', skiprows = 4)
    GDP['Country Name'] = GDP['Country Name'].apply(grenamecountry)
    GDP = GDP[['Country Name', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    GDP.set_index(['Country Name'], inplace = True)
    
    ScimEn = pd.read_excel('scimagojr-3.xlsx')
    ScimEn = ScimEn[ScimEn['Rank'] < 16]
    
    new_df = pd.merge(ScimEn, energy, how = "inner", left_on = 'Country', right_on = 'Country')
    new_df = pd.merge(new_df, GDP, how = "inner", left_on = 'Country', right_index = True)
    new_df = new_df.set_index('Country')
    
    return new_df

def answer_one():
    return ans()[:15]

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [46]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [3]:
def clean(item):
    if re.search("[0-9]+", item):
        spos = re.search("[0-9]+", item).start()
        return item[:spos]
    if re.search("\(.*\)", item):
        spos = re.search("\(.*\)", item).start()-1
        return item[:spos]
    return item

def renamecountry(item):
    item = clean(item)
    if(item == "Republic of Korea"):
        return "South Korea"
    if(item == "United States of America"):
        return "United States"
    if(item == "United Kingdom of Great Britain and Northern Ireland"):
        return "United Kingdom"
    if(item == "China, Hong Kong Special Administrative Region"):
        return "Hong Kong"
    return item

def grenamecountry(item):
    if(item == "Korea, Rep."):
        return "South Korea"
    if(item == "Iran, Islamic Rep."):
        return "Iran"
    if(item == "Hong Kong SAR, China"):
        return "Hong Kong"
    return item

def ans2():
    energy = pd.read_excel('Energy Indicators.xls', skiprows = 17,  skipfooter = 38)
    energy = energy.drop(energy.columns[[0,1]], axis = 1)
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']
    energy.replace(to_replace = "...", value = np.NaN, inplace = True)
    energy['Energy Supply'] = energy.apply(lambda x : (x['Energy Supply'] * 1000000), axis = 1)
    energy['Country'] = energy['Country'].apply(renamecountry)
    energy.set_index(['Country'], inplace = True)
    
    GDP = pd.read_csv('world_bank.csv', skiprows = 4)
    GDP['Country Name'] = GDP['Country Name'].apply(grenamecountry)
    GDP = GDP[['Country Name', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    GDP.set_index(['Country Name'], inplace = True)
    
    ScimEn = pd.read_excel('scimagojr-3.xlsx')
    ScimEn = ScimEn[ScimEn['Rank'] < 16]
    ScimEn.set_index(['Country'], inplace = True)
    
    
    abco = pd.merge(pd.merge(ScimEn, energy, how = "outer", left_index = True, right_index = True), GDP, how = "outer", left_index = True, right_index = True)
    
    abci = pd.merge(pd.merge(ScimEn, energy, how = "inner", left_index = True, right_index = True), GDP, how = "inner", left_index = True, right_index = True)    
    
    return  len(abco) - len(abci)
def answer_two():
    return ans2()

XLRDError: Excel xlsx file; not supported

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [8]:
def calc(rows):
    req = rows[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    rows['avgGDP'] = np.mean(req)
    return rows

def answer_three():
    Top15 = answer_one()
    Top15 = Top15.apply(calc, axis = 1)
    Top15.sort_values(['avgGDP'], ascending = False, inplace = True)
    return Top15['avgGDP']

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [12]:
def calc(rows):
    req = rows[['2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]
    rows['avgGDP'] = np.mean(req)
    return rows

def answer_four():
    Top15 = answer_one()
    Top15 = Top15.apply(calc, axis = 1)
    Top15.sort_values(['avgGDP'], ascending = False, inplace = True)
    return Top15.iloc[5]['2015'] - Top15.iloc[5]['2006']

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [14]:
def answer_five():
    Top15 = answer_one()
    return np.mean(Top15['Energy Supply per Capita'])

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [16]:
def answer_six():
    Top15 = answer_one()
    percent = np.max(Top15['% Renewable'])
    country = Top15[(Top15['% Renewable'] == percent)].index[0]
    res = (country, percent)
    return res

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [63]:
def answer_seven():
    Top15 = answer_one()
    Top15['ratio'] = Top15['Self-citations']/Top15['Citations']
    max_ratio = np.max(Top15['ratio'])
    country = Top15[(Top15['ratio']) == max_ratio].index[0]
    res = (country, max_ratio)
    return res

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [64]:
def answer_eight():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    return Top15.sort_values(['PopEst'], ascending = False).index[2]

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [19]:
def answer_nine():
    Top15 = answer_one()
    Top15['citperperson'] = (Top15['Citable documents']/(Top15['Energy Supply'] / Top15['Energy Supply per Capita']))
    x = Top15[['Energy Supply per Capita', 'citperperson']]
    return x.corr(method = "pearson").iloc[1][0]

In [ ]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [ ]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [48]:
def answer_ten():
    Top15 = answer_one()
    median = Top15['% Renewable'].median()
    
    def func(item):
        if(item >= median):
            return 1
        return 0

    Top15['flag'] = Top15['% Renewable'].apply(func)
    Top15.sort_values(['Rank'], ascending = True, inplace = True)
    HighRenew = Top15['flag']
    return HighRenew


### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [30]:
def answer_eleven():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']

    ContinentDict  = {'China':'Asia', 
                      'United States':'North America', 
                      'Japan':'Asia', 
                      'United Kingdom':'Europe', 
                      'Russian Federation':'Europe', 
                      'Canada':'North America', 
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}
    x = pd.DataFrame(pd.Series(ContinentDict))
    x = x.reset_index()
    x.columns = ['Country','Continent']
    x

    size = []
    sum_ = []
    mean_ = []
    std_ = []
    continent_ = []

    def scalc(rows):
        rows = rows['Country'].tolist()
        return Top15[(Top15.index.isin(rows))]['PopEst'].sum()

    def mcalc(rows):
        rows = rows['Country'].tolist()
        return Top15[(Top15.index.isin(rows))]['PopEst'].mean()

    def stdcalc(rows):
        rows = rows['Country'].tolist()
        return Top15[(Top15.index.isin(rows))]['PopEst'].std()

    for continent, rows in x.groupby('Continent'):
        continent_.append(continent)
        size.append(rows['Country'].count())
        sum__ = scalc(rows)
        sum_.append(sum__)
        mean__ = mcalc(rows)
        mean_.append(mean__)
        std__ = stdcalc(rows)
        std_.append(std__)


    a = pd.DataFrame()
    a['Continent'] = pd.Series(continent_)
    a['size'] = pd.Series(size)
    a['sum'] = pd.Series(sum_)
    a['mean'] = pd.Series(mean_)
    a['std'] = pd.Series(std_)
    a.set_index(['Continent'], inplace = True)

    return a

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [62]:
def answer_twelve():
    Top15 = answer_one()
    Top15['bins'] = pd.cut(Top15['% Renewable'], 5)

    ContinentDict  = {'China':'Asia', 
                      'United States':'North America', 
                      'Japan':'Asia', 
                      'United Kingdom':'Europe', 
                      'Russian Federation':'Europe', 
                      'Canada':'North America', 
                      'Germany':'Europe', 
                      'India':'Asia',
                      'France':'Europe', 
                      'South Korea':'Asia', 
                      'Italy':'Europe', 
                      'Spain':'Europe', 
                      'Iran':'Asia',
                      'Australia':'Australia', 
                      'Brazil':'South America'}
    x = pd.DataFrame(pd.Series(ContinentDict))
    x = x.reset_index()
    x.columns = ['Country','Continent']
    x = pd.merge(x, Top15, how = "outer", left_on = "Country", right_index = True)
    #print(x[['Continent', 'bins']])
    x.sort_values(['Continent'], ascending = True, inplace = True)
   
    return  x.groupby(['Continent', 'bins']).size()

answer_twelve()

Continent      bins            
Asia           (15.753, 29.227]    1
               (2.212, 15.753]     4
Australia      (2.212, 15.753]     1
Europe         (15.753, 29.227]    3
               (2.212, 15.753]     1
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [32]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']

    def separator(item):
        br = str(item).split('.')
        left = br[0][::-1]
        integer = ""
        i = 0
        while(i < len(left)):
            integer += left[i:i+3]
            integer += ","
            i += 3
        item = integer[:-1][::-1] + "." + br[1]
        return item


    Top15['PopEst'] = Top15['PopEst'].apply(separator)

    return   Top15['PopEst']

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [70]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [ ]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!